In this workflow, we extract and highlight nouns in order to get the important parts of reviews.
Be sure to run using GPU to get significant speed-ups!

In [ ]:
token = "" #@param {type: "string"}

STOPWORDS = [
    "i",
    "me",
    "my",
    "myself",
    "we",
    "our",
    "ours",
    "ourselves",
    "you",
    "you're",
    "you've",
    "you'll",
    "you'd",
    "your",
    "yours",
    "yourself",
    "yourselves",
    "he",
    "him",
    "his",
    "himself",
    "she",
    "she's",
    "her",
    "hers",
    "herself",
    "it",
    "it's",
    "its",
    "itself",
    "they",
    "them",
    "their",
    "theirs",
    "themselves",
    "what",
    "which",
    "who",
    "whom",
    "this",
    "that",
    "that'll",
    "these",
    "those",
    "am",
    "is",
    "are",
    "was",
    "were",
    "be",
    "been",
    "being",
    "have",
    "has",
    "had",
    "having",
    "do",
    "does",
    "did",
    "doing",
    "a",
    "an",
    "the",
    "and",
    "but",
    "if",
    "or",
    "because",
    "as",
    "until",
    "while",
    "of",
    "at",
    "by",
    "for",
    "with",
    "about",
    "against",
    "between",
    "into",
    "through",
    "during",
    "before",
    "after",
    "above",
    "below",
    "to",
    "from",
    "up",
    "down",
    "in",
    "out",
    "on",
    "off",
    "over",
    "under",
    "again",
    "further",
    "then",
    "once",
    "here",
    "there",
    "when",
    "where",
    "why",
    "how",
    "all",
    "any",
    "both",
    "each",
    "few",
    "more",
    "most",
    "other",
    "some",
    "such",
    "no",
    "nor",
    "not",
    "only",
    "own",
    "same",
    "so",
    "than",
    "too",
    "very",
    "s",
    "t",
    "can",
    "will",
    "just",
    "don",
    "don't",
    "should",
    "should've",
    "now",
    "d",
    "ll",
    "m",
    "o",
    "re",
    "ve",
    "y",
    "ain",
    "aren",
    "aren't",
    "couldn",
    "couldn't",
    "didn",
    "didn't",
    "doesn",
    "doesn't",
    "hadn",
    "hadn't",
    "hasn",
    "hasn't",
    "haven",
    "haven't",
    "isn",
    "isn't",
    "ma",
    "mightn",
    "mightn't",
    "mustn",
    "mustn't",
    "needn",
    "needn't",
    "shan",
    "shan't",
    "shouldn",
    "shouldn't",
    "wasn",
    "wasn't",
    "weren",
    "weren't",
    "won",
    "won't",
    "wouldn",
    "wouldn't",
    "everything",
    "many",
]


!pip install -q RelevanceAI==2.4.2
!pip install -q flair==0.11.1

from relevanceai.utils import decode_workflow_token
config = decode_workflow_token(token)

text_fields = [config['text_field']]
auth_token = config['authorizationToken']
model_name = config['model_id']
output_fields = [config['outputField']]
cutoff_probability = config['cutoffProbability']

from relevanceai import Client
client = Client(token=auth_token)
ds = client.Dataset(config['dataset_id'])

from flair.data import Sentence
from flair.models import SequenceTagger
from tqdm.auto import tqdm

tagger = SequenceTagger.load(model_name)

def extract_nouns(text, as_documents=False):
  sentence = Sentence(text)
  tagger.predict(sentence)
  nouns = []
  for entity in sentence.get_spans("np"):
    if entity.tag == "NP" and entity.score >= cutoff_probability and \
      entity.text.lower().strip() not in STOPWORDS:
      if as_documents:
        nouns.append({"noun": entity.text, "score": entity.score})
      else:
        nouns.append(entity.text)
  return nouns

def extract_nouns_from_documents(docs, as_documents=False):
  new_docs = [{"_id": d['_id']} for d in docs]
  for d in tqdm(docs):
      for i, t in enumerate(text_fields):
          value = extract_nouns(
              client.get_field(t, d)[:200], 
              as_documents=as_documents
          )
          client.set_field(output_fields[i], new_docs[i], value)
  return new_docs

filters = []
for output_field in output_fields:
  filters += ds[output_field].not_exists()

for chunk in ds.chunk_dataset(select_fields=text_fields, filters=filters):
  docs = extract_nouns_from_documents(chunk)
  ds.upsert_documents(docs)
